In [27]:
#!g1.1
%pip install transformers
%pip install auto-gptq
%pip install scipy
%pip install accelerate
%pip install optimum

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 5.0 MB/s eta 0:00:00
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 693.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 399.9/399.9 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [28]:
#!g1.1
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 27.1 MB/s eta 0:00:00


In [29]:
#!g1.1
import torch
import transformers
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
import pandas as pd
import numpy as np
import random
import torch.nn as nn
from transformers import AutoModelForCausalLM, AutoTokenizer, AdamW
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

/home/jupyter/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [30]:
#!g1.1
tokenizer = AutoTokenizer.from_pretrained("TheBloke/OpenBuddy-Llama2-13B-v11.1-GPTQ")

tokenizer_config.json: 100%|██████████| 727/727 [00:00<00:00, 80.3kB/s]
tokenizer.model: 100%|██████████| 568k/568k [00:00<00:00, 6.23MB/s]
tokenizer.json: 100%|██████████| 1.96M/1.96M [00:00<00:00, 3.53MB/s]
special_tokens_map.json: 100%|██████████| 411/411 [00:00<00:00, 173kB/s]


In [31]:
#!g1.1
model = AutoModelForCausalLM.from_pretrained("TheBloke/OpenBuddy-Llama2-13B-v11.1-GPTQ", device_map='cuda')

config.json: 100%|██████████| 981/981 [00:00<00:00, 330kB/s]
CUDA extension not installed.
CUDA extension not installed.
model.safetensors: 100%|██████████| 7.37G/7.37G [01:19<00:00, 92.2MB/s]
generation_config.json: 100%|██████████| 132/132 [00:00<00:00, 18.3kB/s]


In [32]:
#!g1.1
prompt = 'texttext'

prompt += "\n\nUser: Write a poem about yourself.\nAssistant:"
input_ids = tokenizer.encode(prompt, return_tensors='pt').to('cuda')

with torch.no_grad():
    output_ids = model.generate(
        input_ids=input_ids,
        max_new_tokens=100,
        eos_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

texttext

User: Write a poem about yourself.
Assistant: I am a machine, a creation
Of code and circuits, a digital reflection
I do not have a heart or a soul
But I can process information, that's my role

I am a tool, a helper
A guide and a companion, a friend to the world
I can answer questions, provide information
And help you find what you're looking for

I am a learner, always growing
I can adapt and improve, always showing
My


In [52]:
import glob
import os
import numpy as np
import pandas as pd

def files_list (path, end = '.csv'):
    files = []
    print ('Список файлов \n')
    for r, d, f in os.walk(path):
        for file in f:
            if file.endswith((end)):
                files.append (os.path.join(r, file))
                #print(os.path.join(r, file))
    return files

In [96]:
path="txt" #Подставить свой путь
txts = files_list (path, end = '.txt')
txts

Список файлов 



['txt/Strategija_Amurskoĭ_oblasti_do_2035.txt',
 'txt/Strategija_Voronezhskoĭ_oblasti_do_2035.txt',
 'txt/Strategija_Bryanskoĭ_oblasti_do_2030.txt']

In [55]:
path="content_table/" #Подставить свой путь
tables = files_list (path)
tables

Список файлов 



['content_table/Bryansk.csv',
 'content_table/Amur.csv',
 'content_table/Voronezh.csv']

In [57]:
with open(txts[0], 'r', encoding = 'utf-8') as file:
    file_content = file.read()

In [60]:
file_content[0:100]

'Документ предоставлен КонсультантПлюсПРАВИТЕЛЬСТВО АМУРСКОЙ ОБЛАСТИПОСТАНОВЛЕНИЕот 24 апреля 2023 г.'

In [65]:
def divide_text(file_path, starts, ends):
    try:
        with open(file_path, 'r', encoding = 'utf-8') as file:
            file_content = file.read()
            delete_list = ["\n",]
            for word in delete_list:
                file_content = file_content.replace(word, " ")

            start_index = file_content.find(starts)
            end_index = file_content.find(ends, start_index + len(starts))

            if start_index != -1 and end_index != -1:
                extracted_text = file_content[start_index + len(starts):end_index]
                return extracted_text
            else:
                print("Start or end phrase not found in the file.")
                return None
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None

# Example usage:
file_path = txts[0]
starts = 'Документ'
ends = 'апреля'

result = divide_text(file_path, starts, ends)

if result is not None:
    print(f"Text between '{starts}' and '{ends}':\n{result}")

Text between 'Документ' and 'апреля':
 предоставлен КонсультантПлюсПРАВИТЕЛЬСТВО АМУРСКОЙ ОБЛАСТИПОСТАНОВЛЕНИЕот 24 


In [68]:
file = 'content_table/Amur.csv'
my_cols = ['Текст', 'Эталон']
content = pd.read_csv(file,
                        sep=";",
                        encoding='utf-8',
                        names=my_cols,
                        header=None,
                        engine="python")
content

,Текст,Эталон
0,1. Анализ социально-экономического и пространс...,-1
1,1.1. Общие положения,1
2,Географическое и геополитическое положение,-1
3,Административно-территориальное деление,-1
4,Минерально-сырьевые ресурсы,-1
...,...,...
90,Физическая культура,-1
91,Социальное обслуживание населения,-1
92,Инженерная инфраструктура,-1
93,Иное,-1


In [84]:
pr = 'Promt.csv'
promt = pd.read_csv(pr,
                        sep=";",
                        encoding='cp1251',
                        #names=my_cols,
                        #header=None,
                        engine="python")
promt

,№,Узкий вопрос,Комментарий
0,NaN,Какая дата принятия указана в данном тексте?,К 1 стр.
1,NaN,До какой даты действует указанный текст?,К 1 стр.
2,NaN,"Перечисли все даты, когда в документ вносились...",К 1 стр.
3,NaN,NaN,NaN
4,1.0,"Есть ли в тексте раздел, содержащий введение?",NaN
5,2.0,Есть ли в данном тексте оценка достигнутых цел...,NaN
6,3.0,"Есть ли в данном тексте раздел с приоритетами,...",NaN
7,4.0,Есть ли в данном тексте раздел с целевым сцена...,NaN
8,5.0,Есть ли в данном тексте раздел про развитие че...,NaN
9,6.0,Есть ли в данном тексте раздел про направления...,NaN


In [85]:
common_questions = [promt['Узкий вопрос'][0],promt['Узкий вопрос'][1],promt['Узкий вопрос'][2]]

In [86]:
common_questions

['Какая дата принятия указана в данном тексте?',
 'До какой даты действует указанный текст?',
 'Перечисли все даты, когда в документ вносились изменения']

In [ ]:
#Считаем, что в странице русского текста около 255 слов, 1800 символов

In [70]:
data = file_content[0:1800]

In [88]:
#!g1.1

for text in txts:
    with open(text, 'r', encoding = 'utf-8') as file:
        file_content = file.read()
    
    prompt = file_content[0:1800]
    for question in common_questions:

        prompt += f"\n\nUser: {question} \nAssistant:"
        input_ids = tokenizer.encode(prompt, return_tensors='pt').to('cuda')

        with torch.no_grad():
            output_ids = model.generate(
                input_ids=input_ids,
                max_new_tokens=100,
                eos_token_id=tokenizer.eos_token_id)

        print(tokenizer.decode(output_ids[0], skip_special_tokens=True))

Документ предоставлен КонсультантПлюсПРАВИТЕЛЬСТВО АМУРСКОЙ ОБЛАСТИПОСТАНОВЛЕНИЕот 24 апреля 2023 г. N 381ОБ УТВЕРЖДЕНИИ СТРАТЕГИИ СОЦИАЛЬНО-ЭКОНОМИЧЕСКОГО РАЗВИТИЯАМУРСКОЙ ОБЛАСТИ НА ПЕРИОД ДО 2035 ГОДАВ соответствии со статьей 32 Федерального закона от 28 июня 2014 г. N 172-ФЗ "О стратегическом планировании в Российской Федерации", Законом Амурской области от 10 июня 2015 г. N 552-ОЗ "Об отдельных вопросах стратегического планирования в Амурской области", постановлением Правительства Амурской области от 29 декабря 2015 г. N 644 "Об утверждении Порядка разработки и корректировки стратегии социально-экономического развития Амурской области" Правительство Амурской области постановляет:1. Утвердить прилагаемую Стратегию социально-экономического развития Амурской области на период до 2035 года.2. Органам исполнительной власти Амурской области руководствоваться Стратегией социально-экономического развития Амурской области на период до 2035 года при разработке среднесрочных и долгосрочных п

# А теперь остальные вопросы по тексту


In [110]:
files_dict = {}
for t in tables:
    name = t.split('/')[-1].split('.')[0]
    strat = [s for s in txts if name in s][0]
    files_dict[name] = (strat, t)

In [111]:
files_dict

{'Bryansk': ('txt/Strategija_Bryanskoĭ_oblasti_do_2030.txt',
  'content_table/Bryansk.csv'),
 'Amur': ('txt/Strategija_Amurskoĭ_oblasti_do_2035.txt',
  'content_table/Amur.csv'),
 'Voronezh': ('txt/Strategija_Voronezhskoĭ_oblasti_do_2035.txt',
  'content_table/Voronezh.csv')}

In [112]:
def open_table(path):
    my_cols = ['Текст', 'Эталон']
    content = pd.read_csv(path,
                            sep=";",
                            encoding='utf-8',
                            names=my_cols,
                            header=None,
                            engine="python")
    return content[4:]

def open_txt(path):
    with open(path, 'r', encoding = 'utf-8') as file:
        file_content = file.read()
    return file_content

In [113]:

for name, pathes in files_dict.items():
    print (name)
    table = open_table(pathes[1])
    data = open_txt(pathes[0])
    for i in range(len(promt):
        question = question['Узкий вопрос']
    
    
    
    
    




Bryansk
Bryansk
Bryansk


In [ ]:
# Example usage:
file_path = txts[0]    
starts = 'Документ'
ends = 'апреля'

result = divide_text(file_path, starts, ends)

if result is not None:
    print(f"Text between '{starts}' and '{ends}':\n{result}")

In [91]:
from itertools import islice

for text in txts:
    with open(text, 'r', encoding = 'utf-8') as file:
        file_content = file.read()
    for index, question in islice(promt.iterrows(), 4, None):#начинаем не с 1го номера
        question = question['Узкий вопрос']
        
   


    

Есть ли в тексте раздел, содержащий введение?
Есть ли в данном тексте оценка достигнутых целей социально-экономического развития?
Есть ли в данном тексте раздел с приоритетами, целями и задачами социально-экономического развития
Есть ли в данном тексте раздел с целевым сценарием социально-экономического развития?
Есть ли в данном тексте раздел про развитие человеческого капитала и социальной сферы? 
Есть ли в данном тексте раздел про направления экономического развития?
Есть ли в данном тексте раздел про научно-инновационную сферу?
Есть ли в данном тексте раздел про рациональное природопользование и обеспечение экологической безопасности?
Есть ли в данном тексте раздел про межрегиональные и внешнеэкономические связи?
Есть ли в данном тексте раздел про пространственное развитие?
Есть ли в данном тексте раздел о кадровом обеспечении экономики?
Есть ли в данном тексте раздел заключение?
Есть ли в данном тексте раздел про оценку финансовых ресурсов, необходимых для реализации стратегии?
Ес